In [2]:
import pandas as pd
import os

directory = '../results/simulation/evaluation_jobs'

work_phase_index_to_name = {
    '0': 'balanced',
    '1': 'long-high',
    '2': 'short-high',
}

columns = [
    'filename',
    'scheduling_policy',
    'work_type',
    'work_phases',
    'startup_length',
    'startup_power',
    'waiting_time',
    'id',
    'arrival_time',
    'length',
    'carbon_cost',
    'start_time',
    'waiting_time',
    'exit_time'
]

all_results = pd.DataFrame([])

print(len(os.listdir(directory)))

def read_trace(filename: str): 
        # this is based on the mapping in generate_evaluation_jobs.sh
        parameters = filename.split('_')

        parameter_dict = {
            'scheduling_policy': parameters[0],
            'work_type': parameters[1],
            'work_phases': work_phase_index_to_name[parameters[2]],
            'startup_length': parameters[3],
            'startup_power': parameters[4],
            'waiting_time': parameters[5],
        }

        df = pd.read_csv(f"{directory}/{filename}")
        df = df.drop(df.index[-1])

        if (len(df) < 9):
            print(filename)

        for key, value in parameter_dict.items():
            df[key] = value

        df['filename'] = filename

        return df


# List all files in the directory
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        parameters = filename.split('_')

        if parameters[-1] != 'details':
            continue

        df = read_trace(filename)

        all_results = pd.concat([
            all_results, 
            df
        ], ignore_index=True)

min_delta = 0.1

print(all_results.head())
print(f"Read {len(all_results)} entries")

962
carbon_periodic-phases_1_300_100_96_details
suspend-resume_periodic-phases_2_0_100_96_details
carbon_periodic-phases_2_0_200_12_details
suspend-resume_constant-from-periodic-phases_0_0_100_96_details
carbon_constant-from-periodic-phases_1_300_200_48_details
carbon_constant-from-periodic-phases_0_600_200_6_details
carbon_constant-from-periodic-phases_2_600_200_96_details
suspend-resume_periodic-phases_1_600_100_6_details
carbon_periodic-phases_1_600_100_48_details
carbon_constant-from-periodic-phases_2_0_200_96_details
carbon_constant-from-periodic-phases_1_0_200_48_details
suspend-resume_constant-from-periodic-phases_2_0_200_12_details
suspend-resume_periodic-phases_2_1800_200_6_details
suspend-resume_periodic-phases_2_600_100_96_details
suspend-resume_constant-from-periodic-phases_0_1800_100_48_details
carbon_constant-from-periodic-phases_2_300_200_48_details
carbon_constant-from-periodic-phases_1_300_200_6_details
carbon_constant-from-periodic-phases_2_600_100_96_details
carbon_c

In [3]:
import plotly.express as px
import plotly.io as pio

"""
Let's first compare the same job across different scheduling approaches,
deducing how much carbon is emitted under each scheduler
"""

same_job_different_schedulers = all_results.groupby(["ID", "startup_length", "startup_power", "waiting_time", "work_type", "work_phases", "arrival_time"], sort=True)

same_job_different_schedulers_plot_df = pd.DataFrame([])

had_savings = 0
index = 0

columns_to_keep_for_anova = [
    'scheduling_policy',
    'work_type',
    'work_phases',
    'startup_length',
    'startup_power',
    'waiting_time',
    'length',
    'waiting_time',
]


category_to_int = {}

different_schedulers_total_carbon = pd.DataFrame([])

for category, group_df in same_job_different_schedulers:

    # print(group_df)

    group_df["job_index"] = index

    carbon_cost_non_interrupted = group_df[group_df["scheduling_policy"] == "carbon"]["carbon_cost"].sum()
    carbon_cost_suspend_resume = group_df[group_df["scheduling_policy"] == "suspend-resume"]["carbon_cost"].sum()

    if len(group_df[group_df["scheduling_policy"] == "carbon"]) == 0 or len(group_df[group_df["scheduling_policy"] == "suspend-resume"]) == 0:
        print(category)
        continue

    dummy_dict = group_df[group_df["scheduling_policy"] == "carbon"].iloc[0].to_dict()
    dummy_dict_suspend_resume = group_df[group_df["scheduling_policy"] == "suspend-resume"].iloc[0].to_dict()

    anova_entry = {}
    for key in columns_to_keep_for_anova:
        anova_entry[key] = dummy_dict[key]

    different_schedulers_total_carbon = pd.concat([
        different_schedulers_total_carbon,
        pd.DataFrame([
            {**anova_entry, "carbon_cost":carbon_cost_non_interrupted, "scheduling_policy": "carbon"},
            {**anova_entry, "carbon_cost":carbon_cost_suspend_resume, "scheduling_policy": "suspend-resume"},
        ])
    ])

    x_label = None
    label_key = f"{dummy_dict["startup_length"]},{dummy_dict["startup_power"]},{dummy_dict["work_type"]},{dummy_dict["work_phases"]}"
    if label_key in category_to_int: 
        x_label = category_to_int[label_key]
    else:
        category_to_int[label_key] = len(category_to_int)
        x_label = category_to_int[label_key]

    if (abs(carbon_cost_non_interrupted - carbon_cost_suspend_resume) < min_delta):
        same_job_different_schedulers_plot_df = pd.concat([
            same_job_different_schedulers_plot_df,
            pd.DataFrame([{
                **dummy_dict,
                "job_index": index, 
                "carbon_cost":carbon_cost_non_interrupted, 
                "label": f"Δ < {min_delta}",
                "x_label": x_label
            }])
        ])

    else:
        same_job_different_schedulers_plot_df = pd.concat([
            same_job_different_schedulers_plot_df,
            pd.DataFrame([
                {
                    **dummy_dict,
                    "job_index": index, 
                    "carbon_cost": carbon_cost_non_interrupted, 
                    "label": "without suspending",
                    "x_label": x_label
                },
                {
                    **dummy_dict_suspend_resume,
                    "job_index": index, 
                    "carbon_cost": carbon_cost_suspend_resume, 
                    "length": dummy_dict["length"],
                    "label": "suspend-resume",
                    "x_label": x_label

                }
            ])
        ])

        had_savings += 1

    index += 1

same_job_different_schedulers_plot_df = same_job_different_schedulers_plot_df.sort_values(by="ID")

print(f"Out of {index} groups, {had_savings} had savings between the schedulers")

(np.int64(0), '1800', '200', '12', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(1), '1800', '200', '12', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(2), '1800', '200', '12', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(3), '1800', '200', '12', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(4), '1800', '200', '12', 'periodic-phases', 'balanced', np.int64(0))
Out of 1195 groups, 307 had savings between the schedulers


In [4]:
import plotly.graph_objects as go

"""
Scrolling through by eye, there are some cases where the suspend-resume strategy performed 
better but there are also some cases where it performed worse.

Lets do a graph just plotting each experiment, with the carbon emissions on the y axis
"""

def add_common_styling_to_fig(fig):
    fig.update_layout({
        'margin': dict(l=10, r=10, t=10, b=10)
    })

waiting_times = all_results["waiting_time"].unique()
waiting_time_order = [str(w) for w in sorted([int(w) for w in waiting_times])]


deadlines_color_map = {
    f"Δ < {min_delta}": px.colors.qualitative.Plotly[0],
    'without suspending': px.colors.qualitative.Plotly[1],
    'suspend-resume': px.colors.qualitative.Plotly[2]
}

def make_deadlines_plot(df: pd.DataFrame):
    df = df.reset_index(drop=True)
    df = df.sort_values(by="job_index")

    fig = px.scatter(
        df, x="x_label", y="carbon_cost", color="label",
        hover_data=df.columns, color_discrete_map=deadlines_color_map,
        labels={"label": "Total carbon emissions", "x_label": "Parameter group", "carbon_cost":"gCO₂ emitted"}, facet_row="waiting_time", facet_col="length",
        category_orders={'waiting_time': waiting_time_order}, facet_row_spacing=0.01
    )

    fig.update_layout({
        "height": 1100,
        "width": 800
    })

    label_for_tile={
        "waiting_time=6": "Waiting time 6 hours",
        "waiting_time=12": "Waiting time 12 hours",
        "waiting_time=24": "Waiting time 1 day",
        "waiting_time=48": "Waiting time 2 days",
        "waiting_time=96": "Waiting time 4 days",
    }

    length_label = [
        "1 hour jobs",
        "2 hour jobs",
        "4 hour jobs" ,
        "8 hour jobs",
        "16 hour jobs",   
    ]   

    for i in range(len(length_label)):
        fig.layout['annotations'][i]['text'] = length_label[i]

    fig.for_each_annotation(lambda a: a.update(text = label_for_tile[a.text] if a.text in label_for_tile else a.text))

    fig.update_layout(legend ={
        "yanchor":"top",
        "y":0.99, # equal spacing ;)
        "xanchor":"left",
        "x":0.01
    })

    fig.update_layout({
        'margin': dict(l=10, r=20, t=20, b=10)
    })

    fig.show()

    return fig

same_job_different_schedulers_fig = make_deadlines_plot(same_job_different_schedulers_plot_df)

pio.write_image(same_job_different_schedulers_fig, file="eval_same_job_different_schedulers.pdf")

In [5]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
"""
run an ANOVA, to determine which parameters actually have an effect on the total emissions
"""

target_variable = 'carbon_cost'  # Replace with your target column name

print(different_schedulers_total_carbon.head())

# Iterate through each column in df (excluding the target variable)
results = {}
for column in different_schedulers_total_carbon.columns:
    if column != target_variable:
        # Fit the model
        model = ols(f'{target_variable} ~ C({column})', data=different_schedulers_total_carbon).fit()
        # Perform ANOVA
        anova_table = sm.stats.anova_lm(model, typ=2)
        results[column] = anova_table['PR(>F)'][0]  # Get the p-value

# Convert results to DataFrame
results_df = pd.DataFrame(results.items(), columns=['Feature', 'p_value'])

# Sort by p-value

results_df['p_value (rounded)'] = results_df['p_value'].round(4)
results_df = results_df.sort_values('p_value')
print(results_df)

  scheduling_policy                      work_type work_phases startup_length  \
0            carbon  constant-from-periodic-phases    balanced              0   
1    suspend-resume  constant-from-periodic-phases    balanced              0   
0            carbon  constant-from-periodic-phases   long-high              0   
1    suspend-resume  constant-from-periodic-phases   long-high              0   
0            carbon  constant-from-periodic-phases  short-high              0   

  startup_power waiting_time  length  carbon_cost  
0           100           12    3600         38.4  
1           100           12    3600         38.4  
0           100           12    3600         38.4  
1           100           12    3600         38.4  
0           100           12    3600         28.8  
             Feature       p_value  p_value (rounded)
6             length  0.000000e+00             0.0000
5       waiting_time  1.008635e-23             0.0000
2        work_phases  1.910669e-04     

/tmp/ipykernel_210256/2620312291.py:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_210256/2620312291.py:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_210256/2620312291.py:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_210256/2620312291.py:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

In [6]:
"""
Also try to compare the impact of having phase information
"""

same_job_different_phases = all_results.groupby(["ID", "waiting_time", "startup_length", "scheduling_policy", "startup_power", "scheduling_policy" , "work_phases", "arrival_time"])

same_job_different_phases_df = pd.DataFrame([])
phases_savings_df = pd.DataFrame([])


had_savings = 0
index = 0

min_delta_phases = 1

for category, group_df in same_job_different_phases:

    group_df["job_index"] = index
    carbon_cost_non_constant = group_df[group_df["work_type"] == "periodic-phases"]["carbon_cost"].sum()
    carbon_cost_constant = group_df[group_df["work_type"] == "constant-from-periodic-phases"]["carbon_cost"].sum()

    if len(group_df[group_df["work_type"] == "periodic-phases"]) == 0:
        print(f"missing {category}")
        continue

    dummy_dict = group_df[group_df["work_type"] == "periodic-phases"].iloc[0].to_dict()

    same_job_different_phases_df = pd.concat([
        same_job_different_phases_df,
        pd.DataFrame([{
            **dummy_dict,
            "job_index": index, 
            "carbon_cost":carbon_cost_constant, 
            "difference": round(carbon_cost_non_constant / carbon_cost_constant * 100),
        }])
    ])

    index += 1

print(f"Out of {index} groups, {had_savings} had savings between having phase information")

missing (np.int64(0), '12', '1800', 'suspend-resume', '200', 'suspend-resume', 'balanced', np.int64(0))
missing (np.int64(1), '12', '1800', 'suspend-resume', '200', 'suspend-resume', 'balanced', np.int64(0))
missing (np.int64(2), '12', '1800', 'suspend-resume', '200', 'suspend-resume', 'balanced', np.int64(0))
missing (np.int64(3), '12', '1800', 'suspend-resume', '200', 'suspend-resume', 'balanced', np.int64(0))
missing (np.int64(4), '12', '1800', 'suspend-resume', '200', 'suspend-resume', 'balanced', np.int64(0))
Out of 1195 groups, 0 had savings between having phase information


In [7]:
"""
Plot Carbon-savings form having carbon information or not
"""

label_mapping = {
    "constant-from-periodic-phases": "averaged constant",
    "periodic-phases": "phases"
}

same_job_different_phases_df = same_job_different_phases_df[same_job_different_phases_df["difference"] < 170]

same_job_different_phases_fig = px.scatter(same_job_different_phases_df, x="job_index", y="carbon_cost", 
                                           color="difference", hover_data=same_job_different_phases_df.columns,
                                           labels={"label": "Phases",  "difference": "difference %"}, facet_row="scheduling_policy",
                                            color_continuous_scale=[(0, "green"), (0.1, "grey"), (1, "red")],

                                           range_color=[same_job_different_phases_df["difference"].min(), same_job_different_phases_df["difference"].max()]
)

same_job_different_phases_fig.update_layout({
    "xaxis_title": "Job",
    "yaxis1_title": "gCO₂ emitted",
    "yaxis2_title": "gCO₂ emitted",
})

label_for_phases={
    "scheduling_policy=carbon": "without suspend",
    "scheduling_policy=suspend-resume": "suspend-resume",
}

same_job_different_phases_fig.for_each_annotation(lambda a: a.update(text = label_for_phases[a.text] if a.text in label_for_phases else a.text))

same_job_different_phases_fig.update_layout({
    'margin': dict(l=10, r=10, t=10, b=10)
})

same_job_different_phases_fig.show()

pio.write_image(same_job_different_phases_fig, file="eval_same_job_different_phases.pdf")

In [8]:
"""
Effect of increased waiting times?
"""

import plotly.express as px
import plotly.io as pio

"""
Let's first compare the same job across different scheduling approaches,
deducing how much carbon is emitted under each scheduler
"""

same_job_different_deadlines_df = pd.DataFrame([])

index = 0

waiting_times = all_results["waiting_time"].unique()
print(waiting_times)

deadline_category_to_int = { }

for category, group_df in all_results.groupby(["ID", "startup_length", "startup_power", "scheduling_policy", "work_type", "work_phases", "arrival_time"]):

    group_df["job_index"] = index

    for waiting_time_to_sum in waiting_times:
        # sum up the indivial blocks, in suspend resume, multiple entries will relate to one job
        total_emissions = group_df[group_df["waiting_time"] == waiting_time_to_sum]["carbon_cost"].sum()

        if len(group_df[group_df["waiting_time"] == waiting_time_to_sum]) == 0:
            print(category)
            continue

        x_label = None
        label_key = f"{dummy_dict["startup_length"]},{dummy_dict["startup_power"]},{dummy_dict["work_type"]},{dummy_dict["work_phases"]}"
        if label_key in deadline_category_to_int: 
            x_label = deadline_category_to_int[label_key]
        else:
            deadline_category_to_int[label_key] = len(deadline_category_to_int)
            x_label = deadline_category_to_int[label_key]

        dummy_dict = group_df[group_df["waiting_time"] == waiting_time_to_sum].iloc[0]

        same_job_different_deadlines_df = pd.concat([
            same_job_different_deadlines_df,
            pd.DataFrame([{**dummy_dict, "carbon_cost": total_emissions, "x_label": x_label}])
        ])

    index += 1


['96' '12' '48' '6' '24']
(np.int64(0), '1800', '200', 'suspend-resume', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(1), '1800', '200', 'suspend-resume', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(2), '1800', '200', 'suspend-resume', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(3), '1800', '200', 'suspend-resume', 'periodic-phases', 'balanced', np.int64(0))
(np.int64(4), '1800', '200', 'suspend-resume', 'periodic-phases', 'balanced', np.int64(0))


In [9]:
# some jobs will not have their carbon emissions changed 
# only keep the highest entry per job per emissions

same_job_different_deadlines_df_filtered = pd.DataFrame([])

for category, group_df in same_job_different_deadlines_df.groupby(["job_index", "carbon_cost"]):
    # add the row with the lowest waiting time

    if len(group_df) > 1:
        print(category)

    index_of_lowest = group_df['waiting_time'].idxmin()
    row_with_lowest_wait = group_df.iloc[[index_of_lowest]]

    same_job_different_deadlines_df_filtered = pd.concat([
        same_job_different_deadlines_df_filtered,
        row_with_lowest_wait
    ])

(np.int64(0), np.float64(38.400000000000006))
(np.int64(1), np.float64(38.400000000000006))
(np.int64(2), np.float64(28.80000000000001))
(np.int64(3), np.float64(38.400000000000006))
(np.int64(4), np.float64(38.400000000000006))
(np.int64(5), np.float64(28.800000000000004))
(np.int64(6), np.float64(38.400000000000006))
(np.int64(7), np.float64(38.400000000000006))
(np.int64(8), np.float64(28.80000000000001))
(np.int64(9), np.float64(38.400000000000006))
(np.int64(10), np.float64(38.400000000000006))
(np.int64(11), np.float64(28.800000000000004))
(np.int64(12), np.float64(38.400000000000006))
(np.int64(13), np.float64(38.400000000000006))
(np.int64(14), np.float64(28.80000000000001))
(np.int64(15), np.float64(38.400000000000006))
(np.int64(16), np.float64(38.400000000000006))
(np.int64(17), np.float64(28.800000000000004))
(np.int64(18), np.float64(38.400000000000006))
(np.int64(19), np.float64(38.400000000000006))
(np.int64(20), np.float64(28.80000000000001))
(np.int64(21), np.float64(3

In [13]:
print(same_job_different_deadlines_df_filtered.head())

name_mapping = {
    "6": "6 hours", 
    "12": "12 hours", 
    "24": "1 day", 
    "48": "2 days", 
    "96": "4 days"
}

same_job_different_deadlines_df_filtered['waiting_time'] = same_job_different_deadlines_df_filtered['waiting_time'].astype(str)
same_job_different_deadlines_df_filtered['job_length'] = same_job_different_deadlines_df_filtered['waiting_time'].astype(str)

name_mapping = {
    "6": "6 hours", 
    "12": "12 hours", 
    "24": "1 day", 
    "48": "2 days", 
    "96": "4 days"
}

label_order = list(name_mapping.values())

same_job_different_deadlines_df_filtered['waiting_time_label'] = same_job_different_deadlines_df_filtered['waiting_time'].replace(name_mapping)

waiting_times_ints = [str(w) for w in sorted([int(w) for w in waiting_times])]

def make_deadlines_plot(df: pd.DataFrame):
    fig = px.scatter(
        df, x="x_label", y="carbon_cost", color="waiting_time_label",
        hover_data=df.columns, color_continuous_scale='Aggrnyl', facet_col="ID",
       category_orders={'waiting_time_label': label_order}, labels={"x_label": "Parameter group"}
    )

    fig.update_layout({
        "yaxis_title":"gCO₂ emitted",
    })

    length_label = [
        "1 hour jobs",
        "2 hour jobs",
        "4 hour jobs" ,
        "8 hour jobs",
        "16 hour jobs",   
    ]   

    for i in range(len(length_label)):
        fig.layout['annotations'][i]['text'] = length_label[i]

    fig.update_layout(
        legend_title_text='Waiting times',
        legend=dict(
            x=0.01, 
            y=0.96,
        )
    )


    fig.show()
    return fig

same_jobs_different_deadlines_fig = make_deadlines_plot(same_job_different_deadlines_df_filtered)
pio.write_image(same_jobs_different_deadlines_fig, file="eval_same_job_different_deadlines.pdf")


   ID  arrival_time  length  cpus length_class  resource_class  carbon_cost  \
0   0             0    3600     1          0-2             1.0         27.2   
0   0             0    3600     1          0-2             1.0         38.4   
0   0             0    3600     1          0-2             1.0         76.8   
0   0             0    3600     1          0-2             1.0         27.2   
0   0             0    3600     1          0-2             1.0         38.4   

   dollar_cost  start_time waiting_time  ...  scheduling_policy  \
0       0.0624      291600           96  ...             carbon   
0       0.0624       43200           12  ...             carbon   
0       0.0624       21600            6  ...             carbon   
0       0.0624      291600           96  ...             carbon   
0       0.0624       43200           12  ...             carbon   

                       work_type work_phases startup_length startup_power  \
0  constant-from-periodic-phases    balanced 

In [11]:
from plotly.subplots import make_subplots
from datetime import datetime
import pytz
import sys
import plotly.express as px
from typing import List

sys.path.append('../src/')

import carbon

"""
For debugging purposes, plot some schedules, to see whats up
"""

carbon_trace = carbon.get_carbon_model("DE-hourly-start-july", 0, extra_columns=True).df
start_timestamp = datetime.fromisoformat(carbon_trace["datetime"].min().replace('Z', '+00:00')).timestamp()

carbon_fig = px.line(carbon_trace, x="datetime", y="carbon_intensity_avg")
carbon_fig.update_traces(line=dict(color='black'))

id_to_length = {
    "1": "1",
    "2": "2",
    "3": "4",
    "4": "8",
    "5": "16",
}

def time_to_dates(seconds_since_simulation_start) -> str:
    adjusted_timestamp = seconds_since_simulation_start + start_timestamp
    date = datetime.fromtimestamp(adjusted_timestamp, pytz.timezone('UTC'))
    return date 

def make_gantt(filename: str, color_ids: List[str]):
    df = read_trace(filename)
    df = df.sort_values("ID")

    print(color_ids)
    df["optimal"] = df["ID"].apply(lambda id: "bad" if id in color_ids else "good")

    df["start_time_date"] = df["start_time"].apply(time_to_dates)
    df["submission_date"] = df["arrival_time"].apply(time_to_dates)
    df["exit_time_date"] = df["exit_time"].apply(time_to_dates)
    df["length_hours"] = (df["exit_time"] - df["start_time"]) / 3600

    df["Job length in hours"] = df["ID"].apply(lambda id: id_to_length[str(id+1)])

    fig_gantt = px.timeline(df, x_start="start_time_date", x_end="exit_time_date", y="Job length in hours", hover_data=["start_time", "length_hours"], 
                            color="optimal")

    return fig_gantt

def compare_traces(traces: List[str], waiting_time: int):

    fig = make_subplots(rows=len(traces)+1, cols=1, shared_xaxes=True,
                        subplot_titles=("Scheduling without suspend & resume","Scheduling with suspend & resume"),
                        vertical_spacing=0.1
                        )

    for index, trace in enumerate(traces):
        trace_fig = make_gantt(trace, [] if index == 0 else [4, 3])

        for gantt_trace in trace_fig.data:
            fig.add_trace(gantt_trace, row=index+1, col=1)
            fig.update_layout({
                f"yaxis{index+1}": {'range': [-0.5, 4.5], 
                                    'tickmode': 'linear',
                                    },
                
                'showlegend':False})


    fig.add_trace(carbon_fig.data[0], row=len(traces)+1, col=1)

    fig.update_layout(
        xaxis=dict(
            type='date',
            range=[time_to_dates(0), time_to_dates(waiting_time * 3600 + 3600 * 16)]
        ),
        xaxis2=dict(
            type='date',
            range=[time_to_dates(0), time_to_dates(waiting_time * 3600 + 3600 * 16)]
        ),
        xaxis3=dict(
            type='date',
            range=[time_to_dates(0), time_to_dates(waiting_time * 3600 + 3600 * 16)]
        ),
        height=500,
        width=800
    )

    # fig.update_yaxes(fixedrange=True, row=1, col=1)
    # fig.update_yaxes(fixedrange=True, row=2, col=1)

    fig.add_annotation(x="2024-07-01 01:00:00", y=3.8, col=1, row=2,
            text="Two additional blocks of 15 and 10 minutes",
            showarrow=True,
            arrowhead=1,
            ax=200, ay=10)
    
    fig.add_annotation(x="2024-07-05 07:07:30", y=2.8, col=1, row=2,
            text="Two additional blocks of 15 minutes",
            showarrow=True,
            arrowhead=1,
            ax=-50, ay=35)

    fig.update_layout({
        "yaxis1_title": "Jog length",
        "yaxis2_title": "Job length",
        "yaxis3_title": "gCO₂/kWh"
    })

    fig.update_layout({
        'margin': dict(l=10, r=10, t=20, b=10)
    })

    fig.show()

    return fig


# This is the gantt chart for the mishappen scheduling in the earlier plot
timelimited_gantt = compare_traces([
    "carbon_periodic-phases_1_300_100_96_details",
    "suspend-resume_periodic-phases_1_300_100_96_details"
], 96)

pio.write_image(fig=timelimited_gantt, file="timelimited_gantt.pdf")

carbon_periodic-phases_1_300_100_96_details
[]
[4, 3]


In [12]:
compare_traces([
    "suspend-resume_periodic-phases_2_1800_200_24_details",
    "suspend-resume_constant-from-periodic-phases_2_1800_200_24_details"
], 24)

suspend-resume_periodic-phases_2_1800_200_24_details
[]
suspend-resume_constant-from-periodic-phases_2_1800_200_24_details
[4, 3]
